In [1]:
#Algoritmo 3
#Este algoritmo va a ser una combinacion de medias moviles(mme) y macd. De tal forma que cuando hay un cruce de mme que indique un cambio en la tendencia entraremos a comprar al igual que cuando se produzca una sobre compra en el macd
#Para limitar las perdidas debo bloquear para no operar cuando las mme cuando tengan un cambio de tendencia negativo al igual que en si el macd es negativo
#Solo compro cuando el cruce de mme y el macd sean positivos
#Para salir de la posicion ocurrira cuando el bloqueador se active, qsalte el SL y las pendientes de mme y macd sean negativas
#La distribucion de los pesos vendra determinada por las mme
import requests
import json
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
from datetime import date
#import plotly.graph_objs as go
#import plotly.offline as py
#Se necesita esta linea para que se puedan ver los graficos en el jupyter notebook
#py.init_notebook_mode() #error en que py no esta definido
from api_bme import APIBMEHandler

In [2]:
# Descargo los datos de la api de BME
APIBME = APIBMEHandler('DAX', 'ignaciobermejo_algo3')
maestro_df = APIBME.get_ticker_master()
data_close, data_high, data_low, data_open, data_vol = APIBME.get_data()
benchmark = APIBME.get_close_data_ticker('benchmark')

#almaceno los tickers para el grafico final
returns=data_close
tickers=returns.columns.values #puede que me falte poner de que DF sacarlos

In [3]:
def MME(data,periodo):
    mme_df = data.ewm(alpha = 2/(periodo+1), adjust= False).mean()
    return mme_df

def MACD(data,periodo1,periodo2,periodo3):
    mme_rapida = MME(data,periodo1)
    mme_lenta = MME(data,periodo2)
    signal_rapida = mme_rapida-mme_lenta
    signal_lenta = MME(signal_rapida,periodo3)
    macd_hist = signal_rapida-signal_lenta
    return signal_rapida,signal_lenta,macd_hist


def indice_fuerza(datos,volumen):
    fuerza = datos.diff()*volumen
    mme_fuerza_rapida = MME(fuerza,2)
    mme_fuerza_lenta = MME(fuerza,13)
    return fuerza, mme_fuerza_rapida

In [4]:
#Calculo los retornos diarios
data_returns = np.log(data_close).diff()

#A partitr de los datos diarios los paso a datos semanales
data_close_s = data_close.resample('W').last()
data_high_s = data_high.resample('W').max()
data_low_s = data_low.resample('W').min()
data_open_s = data_open.resample('W').first()
data_vol_s = data_vol.resample('W').sum()

#Aplicar los primeros indicadores semanales de mme y macd
mme_13_s = MME(data_close_s,13)
mme_26_s = MME(data_close_s,26)
signal_rapida_s,signal_lenta_s,macd_hist_s = MACD(data_close_s,12,26,9)

#Filtro las operaciones
penmacd_s = macd_hist_s.diff().iloc[1:,:]/macd_hist_s.iloc[:-1,:]
diff_macd_s = macd_hist_s.diff()
diff_mme_s = mme_13_s.diff()
posicion_s = pd.DataFrame(index=data_close_s.index, columns=data_close_s.columns,data=0)
resta_mme = mme_26_s-mme_13_s
posicion_s [(diff_mme_s>0) | (diff_macd_s>0)] = 1

#Aplicos los filtros para trabajar con el diario
fuerza, mme_fuerza2 = indice_fuerza(data_close,data_vol)
mme_13 = MME(data_close,13)
mme_26 = MME(data_close,26)
signal_rapida,signal_lenta,macd_hist = MACD(data_close,12,26,9)

penmacd = macd_hist.diff().iloc[1:,:]/macd_hist.iloc[:-1,:]
diff_macd = macd_hist.diff()
mme_5 = MME(data_close,5)
diff_mme = mme_5.diff()

posicion = pd.DataFrame(index=data_close.index, columns=data_close.columns,data=0)
posicion [(diff_mme>0) & (diff_macd>0)] = 1

#Tras aplicar los dos filtros que nos permitiran operaciones tenemos que combinarlos
posiciones_diarias = posicion_s.resample('B').ffill(limit=6)
posicion= posicion*posiciones_diarias
posicion = posicion.dropna()
posicion[data_close.isna()] = None

newrow = data_close.index.difference(posicion.index)
posicion_compra_diaria = pd.DataFrame(index=data_close.index.difference(posicion.index), columns=data_close.columns,data=0)
posicion =  posicion_compra_diaria.append(posicion)
posicion_compra_diaria = posicion*1

In [5]:
#Defino el SL minimo del dia como -1.5*atr
aux_1 = data_high - data_low
aux_2 = abs(data_high.iloc[1:,:]-data_close.iloc[0:-1,:])
aux_3 = abs(data_low.iloc[1:,:]-data_close.iloc[0:-1,:])
tro_1 = aux_1[(aux_1>=aux_2) & (aux_1>=aux_3)] 
tro_2 = aux_2[(aux_2>=aux_3) & (aux_2>aux_1)] 
tro_3 = aux_3[(aux_3>aux_2)&(aux_3>aux_1)]
tro_4 = tro_1.add(tro_2, fill_value=0)
tr = tro_4.add(tro_3, fill_value=0)  
atr = tr.rolling(14).mean().ffill(limit=5)
p_venta = data_low-1.5*atr


#Filtro las operaciones de venta semanales
posicion_s[(diff_mme_s<0) & (diff_macd_s<0)] = -1
posicion_venta_diaria = posicion_s.resample('B').ffill(limit=6)
posicion[posicion_venta_diaria==-1] = -1

# Se filtran lasventas diarias
posicion[(p_venta>data_close) | (p_venta>data_low)] =-1
posicion[(mme_26.diff()<0) & (signal_rapida.diff()<0) & (diff_macd<0)] = -1

#Creo el df total de posiciones de compra y venta
posicion[posicion==0] = None
posicion = posicion.ffill()
posicion[posicion == -1] = 0
posicion = posicion.ffill()
posicion[data_close.isna()] = None

In [6]:
#Cuanto comprar
#Ordeno las prioridades de compra segun el incremento porcentual del signal_rapida
mme_12 = MME(data_close,12)
mme_12_pen = mme_12.diff().iloc[1:,:]/mme_12.iloc[:-1,:]
mme_26_pen = mme_26.diff().iloc[1:,:]/mme_26.iloc[:-1,:]
ranker = (mme_12_pen - mme_26_pen)*100
# Definiremos un volumen entre 0.022 y 0.015 por operacion, iniciamos asignando 0.02 en todos los lugares donde tengamos una entrada
# El maximo de entradas en un dia sera de 6  
posicion_compra_diaria[posicion_compra_diaria.diff()!= 1] = 0

for i in range(-2700,-0):
    rankerv = ranker.iloc[i-1,:]
    rankerv [posicion_compra_diaria.iloc[i,:] !=1] = None
    rankerv = rankerv.sort_values(ascending=False)
    rankerv.iloc[0:6] = [0.022,0.02,0.02,0.017,0.015,0.015]
    rankerv.iloc[6:] = 0
    rankerv = rankerv.loc[posicion_compra_diaria.columns]
    posicion_compra_diaria.iloc[i,:] = posicion_compra_diaria.iloc[i,:] * rankerv

posicion_compra_diaria[posicion_compra_diaria == 0] = None
posicion_compra_diaria = posicion_compra_diaria.ffill()
posicion_compra_diaria = posicion_compra_diaria * posicion

posicion_compra_diaria.iloc[-2600:,:].sum(axis=1).max()
vol_inver = posicion_compra_diaria*1
vol_inver[vol_inver.isna()] = 0


In [7]:
def gen_alloc_data(ticker, alloc):
    return {'ticker': ticker,
            'alloc': alloc}
tickers = data_close.columns
tickers = tickers.to_series()
alloc = vol_inver.iloc[-1,:].values

hoy = date.today().strftime('%Y-%m-%d')

allocation = [gen_alloc_data(tickers[i], alloc[i]) for i in np.arange(0,data_close.shape[1])]
APIBME.post_alloc(hoy,allocation)

{'date': '2021-09-01', 'result': True}
